In [18]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions 

import geopandas as gpd
import googlemaps

In [42]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    
    content = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text

    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    interactions_cnt = card.find_element_by_xpath('//a[@class="css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1777fci r-bt1l66 r-1ny4l3l r-bztko3 r-lrvibr"]').text
    
    tweet = (username, handle, postdate, content, reply_cnt, retweet_cnt, like_cnt, interactions_cnt)
    return tweet 

In [40]:


# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)



##################################

##driver.find_element_by_xpath('class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"').click()


##password = driver.find_element_by_xpath("//in")

driver.get('https://twitter.com/login')

sleep(7)

username = driver.find_element_by_xpath("//input[@name='text']")
username.send_keys('GunterICD')


nextB = driver.find_element_by_xpath('//div[@class="css-18t94o4 css-1dbjc4n r-sdzlij r-1phboty r-rs99b7 r-ywje51 r-usiww2 r-2yi16 r-1qi8awa r-1ny4l3l r-ymttw5 r-o7ynqc r-6416eg r-lrvibr r-13qz1uu"]').click()
sleep(5)

password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys('ICD.2023')
sleep(2)

nextB = driver.find_element_by_xpath('//div[@class="css-1dbjc4n r-pw2am6"]/div').click()

C:\Users\elix\AppData\Local\Temp\ipykernel_32188\3089543445.py:4: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [43]:
search_term = input('search term: ')
# navigate to login screen
driver.get('https://twitter.com/search')
driver.maximize_window()
sleep(5)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

search term: from:Cupsfire_gye


In [44]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=114.0.1823.79)


In [46]:
with open('turkcell_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['DisplayName', 'UserName', 'Timestamp', 'Content', 'Comments', 'Likes', 'Retweets','Interactions']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [22]:
import pandas as pd
import numpy as np

df_tweets = pd.read_csv("segura_ep.csv")

df_joan = pd.read_csv("Tweets-delitos2.csv")


In [23]:
df_tweets

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.0,8,"2,314"
1,Segura EP,@segura_ep,2023-09-01T02:46:07.000Z,Desde las cámaras de videovigilancia observamo...,NaN,1.0,7,"2,314"
2,Segura EP,@segura_ep,2023-09-01T02:30:39.000Z,#Alerta | Desde las cámaras de videovigilancia...,NaN,3.0,8,"2,314"
3,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.0,7,"2,314"
4,Segura EP,@segura_ep,2023-09-01T01:27:44.000Z,Desde las cámaras de videovigilancia monitorea...,1.0,NaN,8,"2,314"
...,...,...,...,...,...,...,...,...
531,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,487"
532,Segura EP,@segura_ep,2023-07-10T06:54:34.000Z,Cámaras de videovigilancia @cscgye captan acti...,NaN,1.0,2,"1,487"
533,Segura EP,@segura_ep,2023-07-10T00:30:09.000Z,Cámaras de videovigilancia @cscgye captan el...,NaN,NaN,4,"1,487"
534,Segura EP,@segura_ep,2023-07-09T23:35:59.000Z,#Cuidado | Con el monitoreo de las cámaras de ...,1.0,NaN,7,"1,487"


In [17]:
df_joan

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas
0,Minuto & Medio,@MinMedio,2023-05-13 21:07:56,#Guayaquil \nAccidente de tránsito entre un ca...,101.0,253.000,973,220.8K,1,"-2.1985403506993135, -79.8872444687056"
1,Minuto & Medio,@MinMedio,2023-05-09 19:42:38,#Guayaquil\nUn auto se pasó la luz roja ocasio...,2.0,41.000,199,220.8K,1,"-2.214325924470168, -79.9021528594273"
2,Minuto & Medio,@MinMedio,2023-05-02 17:00:25,#Guayaquil \nUn accidente de tránsito ocurrió ...,NaN,43.000,119,220.8K,1,"-2.1277040608080964, -79.95149150930852"
3,Minuto & Medio,@MinMedio,2023-01-17 15:48:38,#Guayaquil\n\nUn accidente de tránsito se regi...,10.0,32.000,111,30.9K,1,"-2.203267388533896, -79.92337828428693"
4,Minuto & Medio,@MinMedio,2023-01-13 16:30:02,#Guayaquil\nUn ciudadano sufrió un accidente d...,5.0,58.000,146,30.9K,1,"-2.0523766321502097, -79.9462994161406"
...,...,...,...,...,...,...,...,...,...,...
332,Minuto & Medio,@MinMedio,2023-07-16 18:48:04,#Guayaquil _x000D_\nChoferes de buses urbanos ...,44.0,122.000,277,28.3K,1,NaN
333,Minuto & Medio,@MinMedio,2023-07-17 16:45:26,"Como en la década de los 90, vendedores se tom...",634.0,1.135,"1,999",73K,1,NaN
334,Minuto & Medio,@MinMedio,2023-07-18 21:43:43,#Guayaquil _x000D_\nPolicia nacional detuvo a ...,NaN,18.000,124,48K,1,"-2.101462813094633, -79.93364728590731"
335,Minuto & Medio,@MinMedio,2023-07-19 08:04:06,#Guayaquil_x000D_\n3 buses de la metrovia se i...,5.0,34.000,116,48K,1,"-2.1391408737042266, -79.87940332154923"


In [28]:
API_KEY = "" # Token proporcionado por Google
GOOGLE_MAPS_CLIENT = googlemaps.Client(API_KEY)

In [24]:
df_tweets.dropna(subset=["Content"], inplace=True)

In [25]:
df_tweets

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.0,8,"2,314"
1,Segura EP,@segura_ep,2023-09-01T02:46:07.000Z,Desde las cámaras de videovigilancia observamo...,NaN,1.0,7,"2,314"
2,Segura EP,@segura_ep,2023-09-01T02:30:39.000Z,#Alerta | Desde las cámaras de videovigilancia...,NaN,3.0,8,"2,314"
3,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.0,7,"2,314"
4,Segura EP,@segura_ep,2023-09-01T01:27:44.000Z,Desde las cámaras de videovigilancia monitorea...,1.0,NaN,8,"2,314"
...,...,...,...,...,...,...,...,...
531,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,487"
532,Segura EP,@segura_ep,2023-07-10T06:54:34.000Z,Cámaras de videovigilancia @cscgye captan acti...,NaN,1.0,2,"1,487"
533,Segura EP,@segura_ep,2023-07-10T00:30:09.000Z,Cámaras de videovigilancia @cscgye captan el...,NaN,NaN,4,"1,487"
534,Segura EP,@segura_ep,2023-07-09T23:35:59.000Z,#Cuidado | Con el monitoreo de las cámaras de ...,1.0,NaN,7,"1,487"


In [26]:
def get_latitude_longitud(row):
    text = row["Content"].lower().replace("#guayaquil", "") # Borramos el substring '#guayaquil' para que la API pueda recononocer bien el sitio que se menciona en el tweet
    if not text:
        return None
    
    response = GOOGLE_MAPS_CLIENT.geocode(text)
    if response:
        response = response[0]
        return f"{response['geometry']['location']['lat']}, {response['geometry']['location']['lng']}"
    
    return None

In [29]:
df_tweets["coordinates"] = df_tweets.apply(get_latitude_longitud, axis=1)

In [46]:
df_tweets.dtypes

DisplayName      object
UserName         object
Timestamp        object
Content          object
Comments        float64
Likes           float64
Retweets         object
Interactions     object
coordinates      object
dtype: object

In [71]:
df_tweets.dropna(subset=["coordinates"], inplace=True)

In [72]:
df_tweets

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,coordinates
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.0,8,"2,314","-2.1384511, -79.8913772"
1,Segura EP,@segura_ep,2023-09-01T02:46:07.000Z,Desde las cámaras de videovigilancia observamo...,NaN,1.0,7,"2,314","-2.1698488, -79.9097647"
2,Segura EP,@segura_ep,2023-09-01T02:30:39.000Z,#Alerta | Desde las cámaras de videovigilancia...,NaN,3.0,8,"2,314","6.250093, -75.568518"
3,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.0,7,"2,314","-2.230788, -79.8939582"
4,Segura EP,@segura_ep,2023-09-01T01:27:44.000Z,Desde las cámaras de videovigilancia monitorea...,1.0,NaN,8,"2,314","-2.1713035, -79.8912148"
...,...,...,...,...,...,...,...,...,...
530,Segura EP,@segura_ep,2023-07-10T07:15:33.000Z,Por escándalo en vía pública registrado en las...,NaN,2.0,10,"1,487","-2.1941123, -79.8991061"
531,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,487","-2.2438513, -79.8956694"
532,Segura EP,@segura_ep,2023-07-10T06:54:34.000Z,Cámaras de videovigilancia @cscgye captan acti...,NaN,1.0,2,"1,487","-2.1738954, -79.9063997"
533,Segura EP,@segura_ep,2023-07-10T00:30:09.000Z,Cámaras de videovigilancia @cscgye captan el...,NaN,NaN,4,"1,487","-2.2028553, -79.9116221"


In [73]:
keyword_dict = {
    'Robo': ['robo', 'hurto', 'atraco', 'saqueo', 'apropiacion indebida','asalt','roba'],
    'Agresión': ['agresion', 'pelea', 'violencia', 'golpiza', 'ataque', 'maltrato','balacera','hirio','heri','golp','bala','puñetes'],
    'Drogas': ['drogas', 'narcotrafico', 'estupefacientes', 'consumo', 'venta'],
    'Homicidio': ['homicidio', 'asesinato', 'muerte', 'asesino', 'asesinar', 'sicariato','masacre','desmembra'],
    'Accidente Vehicular': ['atropella','accidente', 'colision', 'choque', 'atropello', 'vehiculo', 'conductor','siniestro de transito','volco','volcamiento','volcado'],
    'Secuestro': ['secuestro', 'retenido', 'plagio', 'liberacion', 'rescate'],
    'Fraude': ['fraude', 'estafa', 'engaño', 'falsificacion', 'delito financiero'],
    'Incendio': ['incendio', 'fuego', 'quemar', 'combustion', 'siniestro'],
    'Vandalismo': ['vandalismo', 'destrozo', 'daños', 'destruccion', 'pintadas','explosivo','bomba','relaciones sexuales'],
    'Violación': ['violacion', 'agresion sexual', 'abuso sexual', 'abusar', 'violador'],
    # Agrega más tipos de delitos y palabras clave según sea necesario
}

In [74]:
def remove_tildes(texto):
    tildes_dict = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'Á': 'A',
        'É': 'E',
        'Í': 'I',
        'Ó': 'O',
        'Ú': 'U'
    }

    texto_sin_tildes = ''.join(tildes_dict.get(caracter, caracter) for caracter in texto)
    return texto_sin_tildes

def categorizar_delito(tweet):
    contenido = remove_tildes(tweet['Content'].lower())
    
    # Buscar palabras clave en el contenido del tweet
    for delito, palabras_clave in keyword_dict.items():
        for palabra in palabras_clave:
            if palabra in contenido:
                return delito
    
    # Si no se encuentra ninguna coincidencia, asignar 'Otro'
    return 'Otro'

In [75]:
df_tweets['CategoriaDelito'] = df_tweets.apply(categorizar_delito, axis=1)

In [82]:
df_tweets = df_tweets.loc[df_tweets["CategoriaDelito"]!="Otro"]

In [89]:
df_tweets['Delito'] = 1

C:\Users\elix\AppData\Local\Temp\ipykernel_12764\3377362439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['Delito'] = 1


In [90]:
df_tweets.drop(columns=['CategoriaDelito'], inplace=True)

C:\Users\elix\AppData\Local\Temp\ipykernel_12764\1214050311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets.drop(columns=['CategoriaDelito'], inplace=True)


In [95]:
df_tweets["coordinates"].value_counts()

-1.582611, -79.9622813     20
-2.2438513, -79.8956694    18
-0.1806532, -78.4678382     8
-2.1470677, -79.9117011     5
6.42375, -66.58973          5
                           ..
-2.0494833, -79.9465646     1
-2.1096232, -79.9082425     1
-2.1898222, -79.9427112     1
4.6848788, -74.0625998      1
-2.1555398, -79.9041145     1
Name: coordinates, Length: 128, dtype: int64

In [100]:
df_tweets.loc[df_tweets["coordinates"]=='-2.2438513, -79.8956694']["Content"].values

array(['Desde las cámaras de videovigilancia realizamos el seguimiento por accidente de tránsito en la avenida 25 de Julio.\n\nSe coordinó con miembros de \n@ATM_Transito\n y \n@BomberosGYE\n quienes acudieron al lugar.\n@alcaldiagye\n #GuayaquilSegura',
       '#Alerta | Desde las cámaras de videovigilancia monitoreamos el accidente de tránsito en la av. 25 de Julio y Pio Jaramillo.\n\nSe coordinó con miembros de \n@ATM_Transito\n y \n@BomberosGYE\n, unidades en camino.\n@alcaldiagye\n #GuayaquilSegura',
       '#Alerta | Desde las cámaras de video vigilancia visualizamos a un vehículo reportado como robado  en la av. 25 de Julio, al sur de #Guayaquil. \n@PoliciaDMGZona8\n activó sus unidades y detuvieron al auto y al conductor para investigaciones.\n@alcaldiagye\n#GuayaquilSegura',
       '#Alerta | Cámaras de videovigilancia captan el accidente de tránsito en avenida 25 de Julio y Alberto Avellán Vite. \n\nSe coordinó con miembros de \n@ATM_Transito\n quienes acudieron al sitio. \n@

In [92]:
df_joan

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas
0,Minuto & Medio,@MinMedio,2023-05-13 21:07:56,#Guayaquil \nAccidente de tránsito entre un ca...,101.0,253.000,973,220.8K,1,"-2.1985403506993135, -79.8872444687056"
1,Minuto & Medio,@MinMedio,2023-05-09 19:42:38,#Guayaquil\nUn auto se pasó la luz roja ocasio...,2.0,41.000,199,220.8K,1,"-2.214325924470168, -79.9021528594273"
2,Minuto & Medio,@MinMedio,2023-05-02 17:00:25,#Guayaquil \nUn accidente de tránsito ocurrió ...,NaN,43.000,119,220.8K,1,"-2.1277040608080964, -79.95149150930852"
3,Minuto & Medio,@MinMedio,2023-01-17 15:48:38,#Guayaquil\n\nUn accidente de tránsito se regi...,10.0,32.000,111,30.9K,1,"-2.203267388533896, -79.92337828428693"
4,Minuto & Medio,@MinMedio,2023-01-13 16:30:02,#Guayaquil\nUn ciudadano sufrió un accidente d...,5.0,58.000,146,30.9K,1,"-2.0523766321502097, -79.9462994161406"
...,...,...,...,...,...,...,...,...,...,...
332,Minuto & Medio,@MinMedio,2023-07-16 18:48:04,#Guayaquil _x000D_\nChoferes de buses urbanos ...,44.0,122.000,277,28.3K,1,NaN
333,Minuto & Medio,@MinMedio,2023-07-17 16:45:26,"Como en la década de los 90, vendedores se tom...",634.0,1.135,"1,999",73K,1,NaN
334,Minuto & Medio,@MinMedio,2023-07-18 21:43:43,#Guayaquil _x000D_\nPolicia nacional detuvo a ...,NaN,18.000,124,48K,1,"-2.101462813094633, -79.93364728590731"
335,Minuto & Medio,@MinMedio,2023-07-19 08:04:06,#Guayaquil_x000D_\n3 buses de la metrovia se i...,5.0,34.000,116,48K,1,"-2.1391408737042266, -79.87940332154923"


In [101]:
df_tweets

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,coordinates,Delito
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.0,8,"2,314","-2.1384511, -79.8913772",1
3,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.0,7,"2,314","-2.230788, -79.8939582",1
8,Segura EP,@segura_ep,2023-08-31T01:53:39.000Z,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6,"1,335","-2.2438513, -79.8956694",1
17,Segura EP,@segura_ep,2023-08-29T17:10:33.000Z,Un vehículo reportado como robado fue captado ...,NaN,6.0,20,"2,499","-2.1749333, -79.9417115",1
18,Segura EP,@segura_ep,2023-08-29T08:04:47.000Z,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.0,13,"2,499","-3.9952065, -79.2022123",1
...,...,...,...,...,...,...,...,...,...,...
523,Segura EP,@segura_ep,2023-07-10T08:06:05.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,2.0,8,"1,530","-2.1705075, -79.8794922",1
525,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,530","-2.2438513, -79.8956694",1
527,Segura EP,@segura_ep,2023-07-10T00:30:09.000Z,Cámaras de videovigilancia @cscgye captan el...,NaN,NaN,4,"1,530","-2.2028553, -79.9116221",1
531,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,487","-2.2438513, -79.8956694",1


In [102]:

df_tweets.rename(columns={"coordinates":"Coordenadas"}, inplace=True)

C:\Users\elix\AppData\Local\Temp\ipykernel_12764\3247272436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets.rename(columns={"coordinates":"Coordenadas"}, inplace=True)


In [106]:
df_tweets = df_tweets.loc[:,["DisplayName", "UserName","Timestamp","Content","Comments","Likes","Retweets","Interactions","Delito","Coordenadas"]]

In [107]:
df_tweets

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas
0,Segura EP,@segura_ep,2023-09-01T03:20:23.000Z,Desde las cámaras de videovigilancia observamo...,NaN,2.0,8,"2,314",1,"-2.1384511, -79.8913772"
3,Segura EP,@segura_ep,2023-09-01T02:19:12.000Z,Desde las cámaras de videovigilancia captamos ...,NaN,1.0,7,"2,314",1,"-2.230788, -79.8939582"
8,Segura EP,@segura_ep,2023-08-31T01:53:39.000Z,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6,"1,335",1,"-2.2438513, -79.8956694"
17,Segura EP,@segura_ep,2023-08-29T17:10:33.000Z,Un vehículo reportado como robado fue captado ...,NaN,6.0,20,"2,499",1,"-2.1749333, -79.9417115"
18,Segura EP,@segura_ep,2023-08-29T08:04:47.000Z,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.0,13,"2,499",1,"-3.9952065, -79.2022123"
...,...,...,...,...,...,...,...,...,...,...
523,Segura EP,@segura_ep,2023-07-10T08:06:05.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,2.0,8,"1,530",1,"-2.1705075, -79.8794922"
525,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,530",1,"-2.2438513, -79.8956694"
527,Segura EP,@segura_ep,2023-07-10T00:30:09.000Z,Cámaras de videovigilancia @cscgye captan el...,NaN,NaN,4,"1,530",1,"-2.2028553, -79.9116221"
531,Segura EP,@segura_ep,2023-07-10T06:57:41.000Z,Cámaras de videovigilancia de la @cscgye capta...,NaN,1.0,5,"1,487",1,"-2.2438513, -79.8956694"


In [108]:
df_delitos = pd.concat([df_tweets, df_joan], ignore_index=True)

In [109]:
df_delitos.to_csv('delitos.csv', index=False)